In [91]:
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base #convert python classes into SQL tables
from sqlalchemy import Column, Integer, String, Float, text, Date
import pymysql
from sqlalchemy.orm import Session
pymysql.install_as_MySQLdb()

In [84]:
hawaii_measurements = pd.read_csv("./Resources/hawaii_measurements_clean.csv")
hawaii_stations = pd.read_csv("./Resources/hawaii_stations.csv")

hawaii_measurements=hawaii_measurements[['station','date','prcp','tobs']]

In [99]:
hawaii_measurements.head()
hawaii_stations.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [115]:
hawaii_measurements.dtypes

station     object
date        object
prcp       float64
tobs         int64
dtype: object

In [88]:
Base = declarative_base()

class Measurements(Base):
    
        __tablename__ = 'measurements'
        id = Column(Integer, primary_key=True)
        station = Column(String(255))
        date = Column(Date)
        prcp = Column(Float)
        tobs = Column(Integer)
        
class Stations(Base):
    
        __tablename__ = 'stations'
        id = Column(Integer, primary_key=True)
        station = Column(String(255))
        latitude = Column(Float)
        longitude = Column(Float)
        elevation = Column(Float)

In [120]:
#Create and fill the Measurements table in SQLite

engine=create_engine("sqlite:///../hawaii.sqlite")
session=Session(bind=engine)

#drop the table if it exists
sql = text('DROP TABLE IF EXISTS measurements;')
result = engine.execute(sql)

#Instantiate the table
Base.metadata.create_all(engine)

data = Measurements(
                        id = 0,
                        station = hawaii_measurements.loc[0,'station'],
                        date = datetime.strptime(hawaii_measurements.loc[0,'date'], '%Y-%m-%d'),
                        prcp = hawaii_measurements.loc[0,'prcp'],
                        tobs = int(hawaii_measurements.loc[0,'tobs']) #not sure why I needed to explicitly cast this as an integer but oh well
                        )
session.add(data)
session.commit()

#fill in the rest of the data

for i in range(1,len(hawaii_measurements)):

    data = Measurements(
                            id = i,
                            station = hawaii_measurements.loc[i,'station'],
                            date = datetime.strptime(hawaii_measurements.loc[i,'date'], '%Y-%m-%d'),
                            prcp = hawaii_measurements.loc[i,'prcp'],
                            tobs = int(hawaii_measurements.loc[i,'tobs'])
                            )
    session.add(data)
    session.commit()

In [ ]:
#Create and fill the Stations table in SQLite

engine=create_engine("sqlite:///../hawaii.sqlite")
session=Session(bind=engine)

#drop the table if it exists
sql = text('DROP TABLE IF EXISTS stations;')
result = engine.execute(sql)

#Instantiate the table
Base.metadata.create_all(engine)

data = Stations(
                        id = 0,
                        station = hawaii_stations.loc[0,'station'],
                        latitude = hawaii_stations.loc[0,'latitude'],
                        longitude = hawaii_stations.loc[0,'longitude'],
                        elevation = hawaii_stations.loc[0,'elevation']
                        )
session.add(data)
session.commit()

#fill in the rest of the data

for i in range(1,len(hawaii_stations)):

    data = Stations(
                        id = i,
                        station = hawaii_stations.loc[i,'station'],
                        latitude = hawaii_stations.loc[i,'latitude'],
                        longitude = hawaii_stations.loc[i,'longitude'],
                        elevation = hawaii_stations.loc[i,'elevation']
                            )
    session.add(data)
    session.commit()

In [119]:
# Query the database (testing to make sure it has stuff)
results_list = session.query(Measurements)

for thing in results_list:
    print(thing.tobs)

65


In [46]:
session.close()